In [1]:
# sacado de la documentación, yo no lo he ejecutado.
# %pip install llama-index-llms-openai llama-index-embeddings-openai

# estos son los módulos que tengo instalados
# pip install llama-index
# pip install llama-index-embeddings-huggingface
# pip install llama-index-llms-ollama
# pip install matplotlib


In [1]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "TU KEY"
# openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Response
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    GuidelineEvaluator,
    SemanticSimilarityEvaluator
)
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd

pd.set_option("display.max_colwidth", 0)

In [4]:
# gpt-3.5-turbo
gpt3_5 = OpenAI(temperature=0, model="gpt-3.5-turbo")

faithfulness_gpt3_5 = FaithfulnessEvaluator(llm=gpt3_5)
relevancy_gpt3_5 = RelevancyEvaluator(llm=gpt3_5)
correctness_gpt3_5 = CorrectnessEvaluator(llm=gpt3_5)
semantic = SemanticSimilarityEvaluator()  # coge el modelo de embedings de settings si no se le pasa ninguno.

In [5]:
# creo el modelo de embeddings que quiero probar

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="Alibaba-NLP/gte-Qwen2-1.5B-instruct"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/145k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [6]:
print(Settings.embed_model.model_name)

Alibaba-NLP/gte-Qwen2-1.5B-instruct


In [7]:
# aquí crear el modelo que quieras evaluar

from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3_1_sauerkraut", request_timeout=360.0)

In [8]:
def combinar_diccionarios(dict1, dict2):
    resultado = {}
    for key in dict1:
        if key in dict2:
            # Combina los arrays de ambas claves en un solo array
            resultado[key] = dict1[key] + dict2[key]
    return resultado

In [9]:
# he modificado la del ejemplo para que use result.score en vez de result.passing. en algunos tipos de evaluación sale un resultado más preciso
def get_eval_results(key, eval_results):
    results = eval_results[key]
    sum_score = 0
    count = 0
    for result in results:
        if result.score is not None:  # Verifica si score no es None
            sum_score += result.score
            count += 1
    if count == 0:
        print(f"{key} Score: No valid scores")
        return None
    score = sum_score / count
    print(f"{key} Score: {score}")
    return score

In [10]:
from llama_index.core.llama_dataset import download_llama_dataset, LabelledRagDataset
from llama_index.core.evaluation import BatchEvalRunner

datasets = ["paul_graham", "Blockchain", "Alexnet", "Covid", "Llama2Paper"]
total_results = {"correctness": [], "faithfulness": [], "relevancy": [], "semantic": []}

for dataset_name in datasets:
    path_dataset = f"./eval_data/rag_data/{datasets[0]}/rag_dataset.json"
    path_documents = f"./eval_data/rag_data/{datasets[0]}/source_files"
    rag_dataset = LabelledRagDataset.from_json(path_dataset)
    documents = SimpleDirectoryReader(input_dir=path_documents).load_data()

    rag_dataset_pandas = rag_dataset.to_pandas()
    queries = rag_dataset_pandas["query"]
    reference_answers = rag_dataset_pandas["reference_answer"]
    
    queries = queries[:20]
    reference_answers = reference_answers[:20].to_list()

    splitter = SentenceSplitter(chunk_size=512)
    vector_index = VectorStoreIndex.from_documents(
        documents, transformations=[splitter]
    )

    # ponemos todos los evaluadores que queremos usar aquí.
    runner = BatchEvalRunner(
        {"correctness": correctness_gpt3_5,
         "faithfulness": faithfulness_gpt3_5,
         "relevancy": relevancy_gpt3_5,
         "semantic": semantic},
        workers=8,
    )
    
    # he corregido una cosa de aquí del ejemplo, que no estaba (llm=llm), supongo que por error.
    eval_results = await runner.aevaluate_queries(
        vector_index.as_query_engine(llm=llm),
        queries=queries,
        reference=reference_answers,
    )
    
    total_results = combinar_diccionarios(total_results, eval_results)
    print(f"length of correctcness results: {len(total_results["correctness"])}")

    print(f"results of dataset {dataset_name} of embeddings model: {Settings.embed_model.model_name}")
    score_faithfulness = get_eval_results("faithfulness", eval_results)
    score_relevancy = get_eval_results("relevancy", eval_results)
    score_correctness = get_eval_results("correctness", eval_results)
    score_semantic = get_eval_results("semantic", eval_results)

print(f"total results of all datasets of embeddings model: {Settings.embed_model.model_name}")
score_faithfulness = get_eval_results("faithfulness", total_results)
score_relevancy = get_eval_results("relevancy", total_results)
score_correctness = get_eval_results("correctness", total_results)
score_semantic = get_eval_results("semantic", total_results)

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


length of correctcness results: 20
results of dataset paul_graham of embeddings model: Alibaba-NLP/gte-Qwen2-1.5B-instruct
faithfulness Score: 1.0
relevancy Score: 0.9
correctness Score: 3.861111111111111
semantic Score: 0.9411313196366148
length of correctcness results: 40
results of dataset Blockchain of embeddings model: Alibaba-NLP/gte-Qwen2-1.5B-instruct
faithfulness Score: 0.95
relevancy Score: 0.85
correctness Score: 3.9
semantic Score: 0.9374627925578796
length of correctcness results: 60
results of dataset Alexnet of embeddings model: Alibaba-NLP/gte-Qwen2-1.5B-instruct
faithfulness Score: 0.95
relevancy Score: 0.9
correctness Score: 4.078947368421052
semantic Score: 0.9424337046168236
length of correctcness results: 80
results of dataset Covid of embeddings model: Alibaba-NLP/gte-Qwen2-1.5B-instruct
faithfulness Score: 0.95
relevancy Score: 0.85
correctness Score: 4.055555555555555
semantic Score: 0.9380275707933338
length of correctcness results: 100
results of dataset Llama

In [11]:
print(eval_results.keys())

print(eval_results["correctness"][0].dict().keys())

print(eval_results["correctness"][0].passing)
print(eval_results["correctness"][0].response)
print(eval_results["correctness"][0].contexts)

dict_keys(['correctness', 'faithfulness', 'relevancy', 'semantic'])
dict_keys(['query', 'contexts', 'response', 'passing', 'feedback', 'score', 'pairwise_source', 'invalid_result', 'invalid_reason'])
True
The author's first experience with a computer was on an IBM 1401, which was located in the basement of his junior high school. He had to type programs on punch cards and then stack them in the card reader to load the program into memory for execution. The language he used was an early version of Fortran.

The author faced challenges using this computer due to its limited capabilities. He didn't have any data stored on punched cards, which meant that most tasks required input from these cards. Without such data, his options were restricted. Additionally, he lacked sufficient mathematical knowledge to write more complex programs or engage in interesting calculations like approximating pi. This led to a lack of memorable programs and experiences during this time.

One of the few notable 

In [12]:
print(eval_results["correctness"][:1])

[EvaluationResult(query='In the essay, the author mentions his early experiences with programming. Describe the first computer he used for programming, the language he used, and the challenges he faced.', contexts=None, response="The author's first experience with a computer was on an IBM 1401, which was located in the basement of his junior high school. He had to type programs on punch cards and then stack them in the card reader to load the program into memory for execution. The language he used was an early version of Fortran.\n\nThe author faced challenges using this computer due to its limited capabilities. He didn't have any data stored on punched cards, which meant that most tasks required input from these cards. Without such data, his options were restricted. Additionally, he lacked sufficient mathematical knowledge to write more complex programs or engage in interesting calculations like approximating pi. This led to a lack of memorable programs and experiences during this tim